
## Data Initializing

In [1]:
import pandas as pd
import numpy as np
import data_handler as dh
import model as md
import time
import matplotlib.pyplot as plt
from matplotlib import rc
from multiprocessing import Pool
import pickle

In [2]:
import importlib
importlib.reload(dh)
importlib.reload(md)

<module 'model' from '/home/aleksey/TransportNet/Stable Dynamic & Beckman/model.py'>

In [3]:
import numba
numba.__version__

'0.54.1'

In [4]:
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)
from platform import python_version
print('python', python_version())
print('numpy', np.__version__)
print('pandas', pd.__version__)
import graph_tool
print('graph_tool', graph_tool.__version__)

/home/aleksey/TransportNet/env/bin/python
3.9.2 (default, Feb 28 2021, 17:03:44) 
[GCC 10.2.1 20210110]
sys.version_info(major=3, minor=9, micro=2, releaselevel='final', serial=0)
python 3.9.2
numpy 1.19.5
pandas 1.3.4
graph_tool 2.43 (commit 9d41331e, Wed Jul 7 15:32:52 2021 +0200)


# Beckmann model

parameter $\mu = 0.25$

In [5]:
beckmann_save = 'beckmann_results/'

In [6]:
net_name = 'Anaheim_net.tntp'
trips_name = 'Anaheim_trips.tntp'

handler = dh.DataHandler()
graph_data = handler.GetGraphData(net_name, columns = ['init_node', 'term_node', 'capacity', 'free_flow_time'])
graph_correspondences, total_od_flow = handler.GetGraphCorrespondences(trips_name)

model = md.Model(graph_data, graph_correspondences, 
                    total_od_flow, mu = 0.25, rho = 0.15)

graph_data['graph_table'].head()

,init_node,init_node_thru,term_node,term_node_thru,capacity,free_flow_time
0,1,False,117,True,9000.0,1.090458
1,2,False,87,True,9000.0,1.090458
2,3,False,74,True,9000.0,1.090458
3,4,False,233,True,9000.0,1.090458
4,5,False,165,True,9000.0,1.090458


In [57]:
epsilons = np.logspace(3,1.5,7)
epsilons
results = {}


In [58]:
def compute(kwargs):
    tic = time.time()
    result = model.find_equilibrium(**kwargs)
    toc = time.time()
    return {'args': kwargs, 'result': result, 'time': toc - tic}

def find_eq_for_list(arg_list):
    pool = Pool(processes=len(arg_list))
    return pool.map(compute, arg_list)

 

In [59]:
arg_list = []
for i in range(10):
    kwargs = {}
    max_iter = 2000 * (i + 1)
    kwargs['solver_kwargs'] = {'max_iter' : max_iter, 'stop_crit': 'max_iter',
                 'verbose' : False, 'verbose_step': 2000, 'save_history' : True}
    kwargs['solver_name'] = 'fwm'
    arg_list.append(kwargs)
local_results = find_eq_for_list(arg_list)


Oracles created...
Frank-Wolfe method...
Oracles created...Oracles created...
Frank-Wolfe method...

Frank-Wolfe method...
Oracles created...
Frank-Wolfe method...
Oracles created...
Frank-Wolfe method...
Oracles created...
Oracles created...Frank-Wolfe method...

Frank-Wolfe method...
Oracles created...
Oracles created...Frank-Wolfe method...Oracles created...


Frank-Wolfe method...
Frank-Wolfe method...


In [60]:
for args, res in zip(arg_list, local_results):
    max_iter = args['solver_kwargs']['max_iter']
    results[args['solver_name'] + str(max_iter)] = res

In [61]:
arg_list = []
prev_labels = []
for i in range(10):
    iters_passed = i * 2000
    
    max_iter = (20000 - iters_passed)
    kwargs = {}
    kwargs['solver_kwargs'] = {'max_iter' : max_iter, 'stop_crit': 'max_iter',
                 'verbose' : False, 'verbose_step': 2000, 'save_history' : True}
    kwargs['solver_name'] = 'ustm'
    kwargs['composite'] = True
    if iters_passed > 0:
        prev_label = 'fwm' + str(iters_passed) if iters_passed > 0 else None
        prev_res = results[prev_label]
        prev_labels.append(prev_label)
        kwargs['t_start'] = prev_res['result']['times']
    else:
        prev_labels.append(None)
    arg_list.append(kwargs)
local_results = find_eq_for_list(arg_list)

for args, res, prev_label in zip(arg_list, local_results, prev_labels):
    max_iter = args['solver_kwargs']['max_iter']
    if prev_label is not None:
        res['prev_res'] = prev_label
        results[prev_label + '+' + args['solver_name'] + str(max_iter)] = res
    else:
        results[args['solver_name'] + str(max_iter)] = res

Composite optimization...
Oracles created...
Universal similar triangles method...
Composite optimization...
Oracles created...
Universal similar triangles method...
Composite optimization...
Oracles created...
Universal similar triangles method...
Composite optimization...
Oracles created...
Universal similar triangles method...
Composite optimization...Composite optimization...

Oracles created...Oracles created...

Universal similar triangles method...Universal similar triangles method...

Composite optimization...
Oracles created...
Universal similar triangles method...
Composite optimization...
Oracles created...
Universal similar triangles method...
Composite optimization...
Oracles created...
Universal similar triangles method...
Composite optimization...
Oracles created...
Universal similar triangles method...


In [62]:
print(results.keys())

dict_keys(['fwm2000', 'fwm4000', 'fwm6000', 'fwm8000', 'fwm10000', 'fwm12000', 'fwm14000', 'fwm16000', 'fwm18000', 'fwm20000', 'ustm20000', 'fwm2000+ustm18000', 'fwm4000+ustm16000', 'fwm6000+ustm14000', 'fwm8000+ustm12000', 'fwm10000+ustm10000', 'fwm12000+ustm8000', 'fwm14000+ustm6000', 'fwm16000+ustm4000', 'fwm18000+ustm2000'])


In [63]:
print([result['time'] for result in local_results])

[1121.3744571208954, 1035.0158138275146, 936.4430425167084, 819.9250628948212, 712.0786974430084, 593.2560458183289, 472.89348435401917, 351.7973268032074, 245.95429849624634, 128.7423129081726]


In [64]:
for label, res in results.items():
    print(label, res['result']['history']['dual_gap'][-1])


fwm2000 0.5884025134146214
fwm4000 0.2896952775772661
fwm6000 0.1920236952137202
fwm8000 0.143653568578884
fwm10000 0.11463516601361334
fwm12000 0.09547636448405683
fwm14000 0.08173747733235359
fwm16000 0.0714496064465493
fwm18000 0.06350704631768167
fwm20000 0.05713712074793875
ustm20000 0.954550716560334
fwm2000+ustm18000 0.36511895316652954
fwm4000+ustm16000 0.14991394756361842
fwm6000+ustm14000 0.09082899731583893
fwm8000+ustm12000 0.09808337455615401
fwm10000+ustm10000 0.09299807134084404
fwm12000+ustm8000 0.10088568599894643
fwm14000+ustm6000 0.10259202471934259
fwm16000+ustm4000 0.1662428139243275
fwm18000+ustm2000 0.3872888626065105


In [65]:
arg_list = []
for i in range(10):
    kwargs = {}
    max_iter = 2000 * (i + 1)
    kwargs['solver_kwargs'] = {'max_iter' : max_iter, 'stop_crit': 'max_iter',
                 'verbose' : False, 'verbose_step': 2000, 'save_history' : True}
    kwargs['solver_name'] = 'ustm'
    kwargs['composite'] = True
    arg_list.append(kwargs)
local_results = find_eq_for_list(arg_list)
for args, res in zip(arg_list, local_results):
    max_iter = args['solver_kwargs']['max_iter']
    results[args['solver_name'] + str(max_iter)] = res

Composite optimization...
Composite optimization...Oracles created...
Universal similar triangles method...

Oracles created...
Universal similar triangles method...
Composite optimization...
Oracles created...
Universal similar triangles method...
Composite optimization...
Oracles created...
Universal similar triangles method...
Composite optimization...
Oracles created...
Universal similar triangles method...
Composite optimization...Composite optimization...
Oracles created...
Oracles created...

Universal similar triangles method...
Universal similar triangles method...Composite optimization...
Oracles created...
Universal similar triangles method...

Composite optimization...
Oracles created...
Universal similar triangles method...
Composite optimization...
Oracles created...
Universal similar triangles method...


In [66]:
arg_list = []
prev_labels = []
for i in range(10):
    iters_passed = i * 2000
    
    max_iter = (20000 - iters_passed)
    kwargs = {}
    kwargs['solver_kwargs'] = {'max_iter' : max_iter, 'stop_crit': 'max_iter',
                 'verbose' : False, 'verbose_step': 2000, 'save_history' : True}
    kwargs['solver_name'] = 'wda'
    kwargs['composite'] = False
    if iters_passed > 0:
        prev_label = 'ustm' + str(iters_passed) if iters_passed > 0 else None
        prev_res = results[prev_label]
        prev_labels.append(prev_label)
        kwargs['t_start'] = prev_res['result']['times']
    else:
        prev_labels.append(None)
    arg_list.append(kwargs)
local_results = find_eq_for_list(arg_list)

for args, res, prev_label in zip(arg_list, local_results, prev_labels):
    max_iter = args['solver_kwargs']['max_iter']
    if prev_label is not None:
        res['prev_res'] = prev_label
        results[prev_label + '+' + args['solver_name'] + str(max_iter)] = res
    else:
        results[args['solver_name'] + str(max_iter)] = res

Non-composite optimization...
Oracles created...
Weighted dual averages method...
Non-composite optimization...
Oracles created...
Weighted dual averages method...
Non-composite optimization...
Oracles created...
Weighted dual averages method...
Non-composite optimization...
Oracles created...
Weighted dual averages method...
Non-composite optimization...
Oracles created...
Weighted dual averages method...
Non-composite optimization...
Oracles created...
Weighted dual averages method...
Non-composite optimization...Non-composite optimization...

Oracles created...Oracles created...

Weighted dual averages method...Weighted dual averages method...

Non-composite optimization...Non-composite optimization...

Oracles created...Oracles created...
Weighted dual averages method...

Weighted dual averages method...


fwm2000 0.5884025134146214
fwm4000 0.2896952775772661
fwm6000 0.1920236952137202
fwm8000 0.143653568578884
fwm10000 0.11463516601361334
fwm12000 0.09547636448405683
fwm14000 0.08173747733235359
fwm16000 0.0714496064465493
fwm18000 0.06350704631768167
fwm20000 0.05713712074793875
ustm20000 0.954550716560334
fwm2000+ustm18000 0.36511895316652954
fwm4000+ustm16000 0.14991394756361842
fwm6000+ustm14000 0.09082899731583893
fwm8000+ustm12000 0.09808337455615401
fwm10000+ustm10000 0.09299807134084404
fwm12000+ustm8000 0.10088568599894643
fwm14000+ustm6000 0.10259202471934259
fwm16000+ustm4000 0.1662428139243275
fwm18000+ustm2000 0.3872888626065105
ustm2000 508.07540621212684
ustm4000 244.59702187823132
ustm6000 113.88388580433093
ustm8000 55.32533074519597
ustm10000 25.139251557644457
ustm12000 12.26837044605054
ustm14000 6.231245838338509
ustm16000 3.1730672670528293
ustm18000 1.7369875865988433
wda20000 1350.342186356429
ustm2000+wda18000 1548.3373840895947
ustm4000+wda16000 1714.0795524155

In [68]:
arg_list = []
prev_labels = []
for i in range(10):
    iters_passed = i * 2000
    
    max_iter = (20000 - iters_passed)
    kwargs = {}
    kwargs['solver_kwargs'] = {'max_iter' : max_iter, 'stop_crit': 'max_iter',
                 'verbose' : False, 'verbose_step': 2000, 'save_history' : True}
    kwargs['solver_name'] = 'wda'
    kwargs['composite'] = False
    if iters_passed > 0:
        prev_label = 'fwm' + str(iters_passed) if iters_passed > 0 else None
        prev_res = results[prev_label]
        prev_labels.append(prev_label)
        kwargs['t_start'] = prev_res['result']['times']
    else:
        prev_labels.append(None)
    arg_list.append(kwargs)
local_results = find_eq_for_list(arg_list)

for args, res, prev_label in zip(arg_list, local_results, prev_labels):
    max_iter = args['solver_kwargs']['max_iter']
    if prev_label is not None:
        res['prev_res'] = prev_label
        results[prev_label + '+' + args['solver_name'] + str(max_iter)] = res
    else:
        results[args['solver_name'] + str(max_iter)] = res

Non-composite optimization...
Oracles created...
Weighted dual averages method...
Non-composite optimization...
Non-composite optimization...
Oracles created...
Oracles created...Weighted dual averages method...

Weighted dual averages method...
Non-composite optimization...
Non-composite optimization...Oracles created...
Weighted dual averages method...

Oracles created...
Weighted dual averages method...
Non-composite optimization...
Oracles created...
Weighted dual averages method...
Non-composite optimization...
Oracles created...
Weighted dual averages method...
Non-composite optimization...Non-composite optimization...
Oracles created...
Oracles created...

Weighted dual averages method...
Weighted dual averages method...
Non-composite optimization...
Oracles created...
Weighted dual averages method...


In [69]:
for label, res in results.items():
    print(label, res['result']['history']['dual_gap'][-1])

fwm2000 0.5884025134146214
fwm4000 0.2896952775772661
fwm6000 0.1920236952137202
fwm8000 0.143653568578884
fwm10000 0.11463516601361334
fwm12000 0.09547636448405683
fwm14000 0.08173747733235359
fwm16000 0.0714496064465493
fwm18000 0.06350704631768167
fwm20000 0.05713712074793875
ustm20000 0.954550716560334
fwm2000+ustm18000 0.36511895316652954
fwm4000+ustm16000 0.14991394756361842
fwm6000+ustm14000 0.09082899731583893
fwm8000+ustm12000 0.09808337455615401
fwm10000+ustm10000 0.09299807134084404
fwm12000+ustm8000 0.10088568599894643
fwm14000+ustm6000 0.10259202471934259
fwm16000+ustm4000 0.1662428139243275
fwm18000+ustm2000 0.3872888626065105
ustm2000 508.07540621212684
ustm4000 244.59702187823132
ustm6000 113.88388580433093
ustm8000 55.32533074519597
ustm10000 25.139251557644457
ustm12000 12.26837044605054
ustm14000 6.231245838338509
ustm16000 3.1730672670528293
ustm18000 1.7369875865988433
wda20000 1350.342186356429
ustm2000+wda18000 1548.3373840895947
ustm4000+wda16000 1714.0795524155

In [70]:
arg_list = []
prev_labels = []
for i in range(10):
    iters_passed = i * 2000
    
    max_iter = (20000 - iters_passed)
    kwargs = {}
    kwargs['solver_kwargs'] = {'max_iter' : max_iter, 'stop_crit': 'max_iter',
                 'verbose' : False, 'verbose_step': 2000, 'save_history' : True}
    kwargs['solver_name'] = 'fwm'
    if iters_passed > 0:
        prev_label = 'ustm' + str(iters_passed) if iters_passed > 0 else None
        prev_res = results[prev_label]
        prev_labels.append(prev_label)
        kwargs['t_start'] = prev_res['result']['times']
    else:
        prev_labels.append(None)
    arg_list.append(kwargs)
local_results = find_eq_for_list(arg_list)

for args, res, prev_label in zip(arg_list, local_results, prev_labels):
    max_iter = args['solver_kwargs']['max_iter']
    if prev_label is not None:
        res['prev_res'] = prev_label
        results[prev_label + '+' + args['solver_name'] + str(max_iter)] = res
    else:
        results[args['solver_name'] + str(max_iter)] = res

Oracles created...
Frank-Wolfe method...Oracles created...

Oracles created...
Frank-Wolfe method...Frank-Wolfe method...

Oracles created...Oracles created...
Frank-Wolfe method...

Frank-Wolfe method...Oracles created...

Frank-Wolfe method...
Oracles created...Oracles created...

Frank-Wolfe method...Frank-Wolfe method...

Oracles created...Oracles created...
Frank-Wolfe method...

Frank-Wolfe method...


In [71]:
for label, res in results.items():
    print(label, res['result']['history']['dual_gap'][-1])

fwm2000 0.5884025134146214
fwm4000 0.2896952775772661
fwm6000 0.1920236952137202
fwm8000 0.143653568578884
fwm10000 0.11463516601361334
fwm12000 0.09547636448405683
fwm14000 0.08173747733235359
fwm16000 0.0714496064465493
fwm18000 0.06350704631768167
fwm20000 0.05713712074793875
ustm20000 0.954550716560334
fwm2000+ustm18000 0.36511895316652954
fwm4000+ustm16000 0.14991394756361842
fwm6000+ustm14000 0.09082899731583893
fwm8000+ustm12000 0.09808337455615401
fwm10000+ustm10000 0.09299807134084404
fwm12000+ustm8000 0.10088568599894643
fwm14000+ustm6000 0.10259202471934259
fwm16000+ustm4000 0.1662428139243275
fwm18000+ustm2000 0.3872888626065105
ustm2000 508.07540621212684
ustm4000 244.59702187823132
ustm6000 113.88388580433093
ustm8000 55.32533074519597
ustm10000 25.139251557644457
ustm12000 12.26837044605054
ustm14000 6.231245838338509
ustm16000 3.1730672670528293
ustm18000 1.7369875865988433
wda20000 1350.342186356429
ustm2000+wda18000 1548.3373840895947
ustm4000+wda16000 1714.0795524155

In [72]:
with open('anaheim_result_new_start_beckmann.pickle', 'wb') as f:
    pickle.dump(results, f)

In [73]:
with open('anaheim_result_new_start_beckmann.pickle', 'rb') as f:
        loaded_results = pickle.load(f)
 

In [ ]:
print(loaded_results)